In [1]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import pandas as pd
from typing import List

import src.data.breathe_data as breathe_data
import src.inference.helpers as ih
import src.inference.long_inf_slicing as slicing
import src.models.builders as mb
import src.models.var_builders as var_builders
import src.models.helpers as mh

## Breathe data processing

In [2]:
df = breathe_data.load_o2_fev1_df_from_excel()
df.head()

,ID,Date Recorded,FEV1,O2 Saturation,ecFEV1,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy
0,101,2019-02-20,1.31,97.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,36.287474,99.767593
1,101,2019-02-21,1.29,96.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,35.733466,98.739061
2,101,2019-02-22,1.32,96.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,36.564477,98.739061
3,101,2019-02-23,1.28,97.0,1.33,53,Male,173.0,3.610061,97.22596,36.841481,35.456463,99.767593
4,101,2019-02-24,1.33,98.0,1.36,53,Male,173.0,3.610061,97.22596,37.672492,36.841481,100.796125


In [3]:
# Create factor graph
height = 170
age = 30
sex = "Male"
(
    HFEV1,
    ecFEV1,
    AR,
    HO2Sat,
    O2SatFFA,
    IA,
    UO2Sat,
    O2Sat,
) = var_builders.o2sat_fev1_point_in_time(height, age, sex)

var_to_colname = {
    ecFEV1.name: "ecFEV1",
    O2Sat.name: "O2 Saturation",
}

df[ecFEV1.name] = df.apply(
    lambda row: ecFEV1.get_bin_for_value(row[var_to_colname[ecFEV1.name]])[1], axis=1
)
df[O2Sat.name] = df.apply(
    lambda row: O2Sat.get_bin_for_value(row[var_to_colname[O2Sat.name]])[1], axis=1
)
df.head()

,ID,Date Recorded,FEV1,O2 Saturation,ecFEV1,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,ecFEV1 (L),O2 saturation (%)
0,101,2019-02-20,1.31,97.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,36.287474,99.767593,26,47
1,101,2019-02-21,1.29,96.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,35.733466,98.739061,26,46
2,101,2019-02-22,1.32,96.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,36.564477,98.739061,26,46
3,101,2019-02-23,1.28,97.0,1.33,53,Male,173.0,3.610061,97.22596,36.841481,35.456463,99.767593,26,47
4,101,2019-02-24,1.33,98.0,1.36,53,Male,173.0,3.610061,97.22596,37.672492,36.841481,100.796125,27,48


## Slicing algorithm

In [4]:
# Algorithm used to select the stopping criteria
# Moved to src.inference.long_inf_slicing and updated to be general to any shared variables input

# def query_across_days(
#     df,
#     belief_propagation,
#     shared_variables: List[SharedNodeVariable],
#     variables: List[str],
#     n_epochs,
# ):
#     epoch = 0
#     df_res_hfev1 = pd.DataFrame(index=HFEV1.bins_str)

#     df_res = pd.DataFrame(
#         columns=["Epoch"] + list(map(lambda v: v.name, shared_variables))
#     )

#     post_hfev1_old_epoch = HFEV1._uniform_prior()
#     post_ho2sat_old_epoch = HO2Sat._uniform_prior()
#     while True:
#         print(f"epoch {epoch}")

#         post_hfev1_old_day = HFEV1._uniform_prior()
#         post_ho2sat_old_day = HO2Sat._uniform_prior()
#         diffs_hfev1_day = np.array([])
#         diffs_ho2sat_day = np.array([])
#         for i in range(len(df)):
#             day = df["Date Recorded"].iloc[i].strftime("%Y-%m-%d")

#             def build_evidence(variables):
#                 evidence = {}
#                 for variable in variables:
#                     idx_obs = df[variable].iloc[i]
#                     evidence[variable] = idx_obs
#                 return evidence

#             evidence = build_evidence(variables)

#             def build_virtual_evidence(shared_variables):
#                 virtual_evidence = {}
#                 for shared_var in shared_variables:
#                     virtual_message = shared_var.get_virtual_message(day)
#                     if virtual_message is not None:
#                         virtual_evidence[shared_var.name] = virtual_message
#                 return virtual_evidence

#             virtual_evidence = build_virtual_evidence(shared_variables)

#             var_to_infer = list(map(lambda v: v.name, shared_variables))

#             # Query the graph
#             res, messages = belief_propagation.query(
#                 var_to_infer, evidence, virtual_evidence, get_messages=True
#             )

#             # Save message for current day
#             for shared_var in shared_variables:
#                 shared_var.add_message(day, messages[shared_var.graph_key])

#             post_hfev1_old_day, diff_hfev1_day = get_diff(
#                 res, post_hfev1_old_day, HFEV1
#             )
#             post_ho2sat_old_day, diff_ho2sat_day = get_diff(
#                 res, post_ho2sat_old_day, HO2Sat
#             )
#             # print(
#             #     f"Epoch {epoch}, day {i} - Diff hfev1 {diff_hfev1_day}, diff ho2sat {diff_ho2sat_day}"
#             # )
#             diffs_hfev1_day = np.append(diffs_hfev1_day, post_ho2sat_old_day)
#             diffs_ho2sat_day = np.append(diffs_ho2sat_day, diff_ho2sat_day)

#         # print(
#         #     f"Epoch {epoch} - Sum daily diffs for HFEV1: {diffs_hfev1_day.sum()}, and HO2Sat: {diffs_ho2sat_day.sum()}"
#         # )
#         post_hfev1_old_epoch, diff_hfev1_epoch = get_diff(
#             res, post_hfev1_old_epoch, HFEV1
#         )
#         post_ho2sat_old_epoch, diff_ho2sat_epoch = get_diff(
#             res, post_ho2sat_old_epoch, HO2Sat
#         )
#         print(
#             f"Epoch {epoch} - Posteriors' diff for HFEV1: {diff_hfev1_epoch}, and HO2Sat: {diff_ho2sat_epoch}"
#         )

#         # Create new row df with epoch, and on shared variables array per row cel
#         new_row = [epoch] + list(map(lambda v: res[v.name].values, shared_variables))
#         # Same but as df
#         new_row = pd.DataFrame(
#             [new_row], columns=["Epoch"] + list(map(lambda v: v.name, shared_variables))
#         )

#         df_res = pd.concat([df_res, new_row], ignore_index=True)
#         df_res_hfev1[f"{epoch}"] = res[HFEV1.name].values

#         if epoch >= n_epochs:
#             return df_res, df_res_hfev1
#         epoch += 1


# def get_diff(res, old, var):
#     new = res[var.name].values
#     diff = np.abs(new - old).sum()
#     return new, diff

In [5]:
# Meta
key_hfev1 = f"['{ecFEV1.name}', '{HFEV1.name}', '{AR.name}'] -> {HFEV1.name}"
key_ho2sat = f"['{O2SatFFA.name}', '{HO2Sat.name}', '{AR.name}'] -> {HO2Sat.name}"

HFEV1shared = slicing.SharedNodeVariable(HFEV1.name, len(HFEV1.bins), key_hfev1)
HO2Satshared = slicing.SharedNodeVariable(HO2Sat.name, len(HO2Sat.bins), key_ho2sat)

vars = [AR, IA]
shared_vars = [HFEV1shared, HO2Satshared]
obs_vars = [ecFEV1.name, O2Sat.name]

# Specific to ID 101
df_for_ID = df[df["ID"] == "107"]


def infer_and_plot_for_id(df_for_ID, shared_vars, vars, obs_vars, diff_threshold=1e-8):
    df_for_ID = df_for_ID.reset_index(drop=True)
    print(f"\nID: {df_for_ID.ID.iloc[0]}")
    print(f"Amount of data: {len(df_for_ID)}")

    height = df_for_ID.Height.iloc[0]
    age = df_for_ID.Age.iloc[0]
    sex = df_for_ID.Sex.iloc[0]
    model, inf_alg, HFEV1, ecFEV1, AR, HO2Sat, O2SatFFA, IA, UO2Sat, O2Sat = (
        mb.o2sat_fev1_point_in_time_model_2(height, age, sex)
    )

    df_query_res, df_res_epoch = slicing.query_across_days(
        df_for_ID, inf_alg, shared_vars, vars, obs_vars, diff_threshold
    )

    save = True

    slicing.plot_posterior_validation(
        df_query_res,
        shared_vars[0],
        HFEV1,
        shared_vars[1],
        HO2Sat,
        df_for_ID,
        ecFEV1,
        O2Sat,
        n_epochs=len(df_res_epoch),
        colorscale=[[0, "white"], [0.33, "yellow"], [0.66, "green"], [1, "blue"]],
        save=save,
    )

    slicing.plot_query_res(
        df_for_ID, ecFEV1, O2Sat, df_query_res, AR, IA, HFEV1, HO2Sat, save=save
    )


# infer_and_plot_for_id(df_for_ID, shared_vars, vars, obs_vars, diff_threshold=1e-8)
df.groupby("ID").apply(
    lambda df_for_ID: infer_and_plot_for_id(
        df_for_ID, shared_vars, vars, obs_vars, diff_threshold=1e-8
    )
)


ID: 101
Amount of data: 816
epoch 0
Epoch 0 - Posteriors' diff for Healthy FEV1 (L): 1.9586847380467245
Epoch 0 - Posteriors' diff for Healthy O2 saturation (%): 1.8218215218269185
epoch 1
Epoch 1 - Posteriors' diff for Healthy FEV1 (L): 2.5455991805152083e-11
Epoch 1 - Posteriors' diff for Healthy O2 saturation (%): 1.079598686380357e-12
All diffs are below 1e-08, running another epoch to get all posteriors
epoch 2
Epoch 2 - Posteriors' diff for Healthy FEV1 (L): 0.0
Epoch 2 - Posteriors' diff for Healthy O2 saturation (%): 0.0

ID: 102
Amount of data: 151
epoch 0
Epoch 0 - Posteriors' diff for Healthy FEV1 (L): 1.9696580781445796
Epoch 0 - Posteriors' diff for Healthy O2 saturation (%): 1.8717417893952493
epoch 1
Epoch 1 - Posteriors' diff for Healthy FEV1 (L): 6.352747104407253e-22
Epoch 1 - Posteriors' diff for Healthy O2 saturation (%): 0.0
All diffs are below 1e-08, running another epoch to get all posteriors
epoch 2
Epoch 2 - Posteriors' diff for Healthy FEV1 (L): 0.0
Epoch 2 -

/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/inference/long_inf_slicing.py:34: RuntimeWarning:

invalid value encountered in divide



Epoch 0 - Posteriors' diff for Healthy FEV1 (L): nan
Epoch 0 - Posteriors' diff for Healthy O2 saturation (%): nan
epoch 1
Epoch 1 - Posteriors' diff for Healthy FEV1 (L): nan
Epoch 1 - Posteriors' diff for Healthy O2 saturation (%): nan
epoch 2
Epoch 2 - Posteriors' diff for Healthy FEV1 (L): nan
Epoch 2 - Posteriors' diff for Healthy O2 saturation (%): nan
epoch 3
Epoch 3 - Posteriors' diff for Healthy FEV1 (L): nan
Epoch 3 - Posteriors' diff for Healthy O2 saturation (%): nan
epoch 4
Epoch 4 - Posteriors' diff for Healthy FEV1 (L): nan
Epoch 4 - Posteriors' diff for Healthy O2 saturation (%): nan
epoch 5
Epoch 5 - Posteriors' diff for Healthy FEV1 (L): nan
Epoch 5 - Posteriors' diff for Healthy O2 saturation (%): nan
epoch 6
Epoch 6 - Posteriors' diff for Healthy FEV1 (L): nan
Epoch 6 - Posteriors' diff for Healthy O2 saturation (%): nan
epoch 7
Epoch 7 - Posteriors' diff for Healthy FEV1 (L): nan
Epoch 7 - Posteriors' diff for Healthy O2 saturation (%): nan
epoch 8
Epoch 8 - Poster

KeyboardInterrupt: 